In [2]:
%matplotlib inline

import pickle
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import tensorflow as tf
from utils import accuracy
from inspect import signature

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
from preprocess import Preprocess
from training import Server

## Compute metrics: AP, TPR, FPR, Precision

In [4]:
# Get results
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import pickle

In [11]:
LINE_WIDTH = 2.5
XLIM = [0, 1]
YLIM = [0, 1.05]
def print_pred(h5_path):
    t = Server(pred_only=True, h5_path=h5_path)
    props,y_truth = t.predict()
    print("Testing on {} pairs with {} correlated ones".format(len(y_truth), np.count_nonzero(y_truth == 1)))
    plot_hist(props, y_truth)
    plot_roc(props, y_truth)
    plot_pr(props, y_truth)
    plot_tpr_fpr(props, y_truth)
    
def plot_hist(props, y_truth):
    print('y_truth: ', y_truth[30:54])
    print('y_pred: ', (tf.sigmoid(props) >= 0.5).numpy().astype(float)[30:54])
    plt.figure()
    fig,axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].hist(y_truth)
    axs[1].hist((tf.sigmoid(props) >= 0.5).numpy().astype(float))
    axs[0].set_title('y_truth')
    axs[1].set_title('y_pred')
    axs[0].grid(alpha=0.5)
    axs[1].grid(alpha=0.5)
    plt.grid()
    plt.show()
    return

def plot_roc(props, y_truth, plot=True):
    scores = tf.sigmoid(props).numpy().astype(float)
    fpr, tpr, thresholds = metrics.roc_curve(y_truth, scores, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    if plot:
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange',
                 label=f'ROC curve (auc = {auc:.3f})',
                 linewidth=LINE_WIDTH)
        plt.plot([0, 1], [0, 1], linestyle='--', label="Baseline",
                 linewidth=LINE_WIDTH)
        plt.xlim(XLIM)
        plt.ylim(YLIM)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic Curve')
        plt.legend()
        plt.grid()
        plt.show()
    return fpr, tpr, thresholds


def plot_pr(props, y_truth, bs=0.5, plot=True):
    scores = tf.sigmoid(props).numpy().astype(float)
    average_precision = average_precision_score(y_truth, scores)
    pr, rec, thresholds = precision_recall_curve(y_truth, scores, pos_label=1)
    if plot:
        plt.figure()
        plt.plot(rec, pr, color='darkgreen',
                 label=f'PR curve (AP = {average_precision:.3f})',
                 linewidth=LINE_WIDTH)
        plt.plot([0, 1], [bs, bs], color='blue', linestyle='--', label="Baseline",
                 linewidth=LINE_WIDTH)
        #plt.xlim([-0.05, 1.05])
        #plt.ylim([-0.05, 1.05])
        plt.xlim(XLIM)
        plt.ylim(YLIM)
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.grid()
        plt.show()
    return pr, rec, thresholds


def plot_new_pr(pr, rec):
    plt.figure()
    plt.plot(rec, pr, color='green',
             label=f'PR curve',
             linewidth=LINE_WIDTH)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Opt Precision-Recall Curve')
    plt.legend()
    plt.grid()
    plt.show()
    return


def plot_tpr_fpr(props, y_truth, markevery=100, log_scale=True):
    scores = tf.sigmoid(props).numpy().astype(float)
    fpr, tpr, thresholds = metrics.roc_curve(y_truth, scores, pos_label=1)
    plt.figure()
    plt.plot(thresholds, tpr, color='black',
             label=f'TP', marker='s', markevery=markevery,
             linewidth=LINE_WIDTH)
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    plt.xlim(XLIM)
    plt.ylim(YLIM)
    plt.xlabel('Threshold')
    #plt.ylabel('True Positive Rate')
    plt.legend()
    plt.grid()
    plt.show()
    
    plt.figure()
    plt.plot(thresholds, fpr, color='red',
             label=f'FP', marker='s', markevery=markevery,
             linewidth=LINE_WIDTH)
    if log_scale:
        plt.yscale('log')
    plt.xlim([0.0, 1.0])
    plt.xlabel('Threshold')
    plt.ylabel('Log scale')
    plt.legend()
    plt.grid()
    plt.show()
    return fpr, tpr, thresholds


def plot_new_tpr_fpr(tpr, fpr, thresholds, markevery=100, log_scale=True):
    plt.figure()
    plt.plot(thresholds, tpr, color='black',
             label=f'TP', marker='s', markevery=markevery,
             linewidth=LINE_WIDTH)
    plt.xlabel('Threshold')
    #plt.ylabel('True Positive Rate')
    plt.legend()
    plt.grid()
    plt.show()
    
    plt.figure()
    plt.plot(thresholds, fpr, color='red',
             label=f'FP', marker='s', markevery=markevery,
             linewidth=LINE_WIDTH)
    if log_scale:
        plt.yscale('log')
    #plt.xlim([0.0, 1.0])
    plt.xlabel('Threshold')
    plt.ylabel('Log scale')
    plt.legend()
    plt.show()
    return


def optimize(fpr, tpr, thresholds, pr, rec, reach_tpr=0.8, reach_fpr=0.001, markevery=10):
    good_tpr = []
    good_fpr = []
    good_thresh = []

    # Optimize for TPR
    deepcorr_tpr = reach_tpr
    optim_tpr = deepcorr_tpr
    fpr_optim_tpr = 1.0

    # Optimize for FPR
    deepcorr_fpr = reach_fpr
    optim_fpr = deepcorr_fpr
    tpr_optim_fpr = 0.0


    for i in range(len(tpr)):
        if abs(tpr[i] - optim_tpr) < 0.01 and abs(tpr[i] - deepcorr_tpr) < 0.01 and fpr[i] < fpr_optim_tpr:
            optim_tpr = tpr[i]
            fpr_optim_tpr = fpr[i]
        if abs(fpr[i] - optim_fpr) < 0.0001 and abs(fpr[i] - deepcorr_fpr) < 0.0001 and tpr[i] > tpr_optim_fpr:
            optim_fpr = fpr[i]
            tpr_optim_fpr = tpr[i]
        if tpr[i] > 0.4 and fpr[i] < 0.01:
            good_tpr += [tpr[i]]
            good_fpr += [fpr[i]]
            good_thresh += [thresholds[i]]


    good_pr = []
    good_rec = []

    # Optimize for Precision
    optim_pr = 0.6
    rec_optim_pr = 0.0

    # Optimize for Recall
    optim_rec = 0.6
    pr_optim_rec = 0.0

    for i in range(len(pr)):
        if pr[i] > optim_pr and rec[i] > rec_optim_pr:
            optim_pr = pr[i]
            rec_optim_pr = rec[i]
        if rec[i] > optim_rec and pr[i] > pr_optim_rec:
            optim_rec = rec[i]
            pr_optim_rec = pr[i]
        if pr[i] > 0.4 and rec[i] > 0.4:
            good_pr += [pr[i]]
            good_rec += [rec[i]]

    return tpr_optim_fpr, optim_fpr, optim_tpr, fpr_optim_tpr

In [19]:
results_dir = "results/"

folds = range(5)
runs = [10]

M_resfiles = []
for run in runs:
    for fold in folds:
        M_resfiles += [results_dir + f"test_tc_M_flowlen-100_neg-49_NtimesN-fold{fold}-run{run}.pkl"]
        
print(M_resfiles)

S_resfiles = []
for run in runs:
    for fold in folds:
        S_resfiles += [results_dir + f"test_tc_S_flowlen-100_neg-49_NtimesN-fold{fold}-run{run}.pkl"]

print(S_resfiles)

['results/test_tc_M_flowlen-100_neg-49_NtimesN-fold0-run10.pkl', 'results/test_tc_M_flowlen-100_neg-49_NtimesN-fold1-run10.pkl', 'results/test_tc_M_flowlen-100_neg-49_NtimesN-fold2-run10.pkl', 'results/test_tc_M_flowlen-100_neg-49_NtimesN-fold3-run10.pkl', 'results/test_tc_M_flowlen-100_neg-49_NtimesN-fold4-run10.pkl']
['results/test_tc_S_flowlen-100_neg-49_NtimesN-fold0-run10.pkl', 'results/test_tc_S_flowlen-100_neg-49_NtimesN-fold1-run10.pkl', 'results/test_tc_S_flowlen-100_neg-49_NtimesN-fold2-run10.pkl', 'results/test_tc_S_flowlen-100_neg-49_NtimesN-fold3-run10.pkl', 'results/test_tc_S_flowlen-100_neg-49_NtimesN-fold4-run10.pkl']


In [17]:
AP_M = []
PR_for_FPR_M = []
PR_for_TPR_M = []
TPR_for_TPR_M = []
FPR_for_TPR_M = []
FPR_for_FPR_M = []
TPR_for_FPR_M = []

for resfile in M_resfiles:
    print(resfile)
    f = open(resfile, "rb")
    props, y_truth = pickle.load(f)

    av_prec = average_precision_score(y_truth, props)

    AP_M += [av_prec]

    pr, rec, _ = plot_pr(props, y_truth, plot=False)
    fpr, tpr, thresholds = plot_roc(props, y_truth, plot=False)
    tpr_optim_fpr, optim_fpr, optim_tpr, fpr_optim_tpr = optimize(fpr, tpr, thresholds, pr, rec)

    n_pos = len([y for y in y_truth if y == 1])
    n_neg = len([y for y in y_truth if y == 0])

    tp1 = optim_tpr * n_pos
    fp1 = fpr_optim_tpr * n_neg
    Pr1 = tp1 / (tp1 + fp1)

    tp2 = tpr_optim_fpr * n_pos
    fp2 = optim_fpr * n_neg
    Pr2 = tp2 / (tp2 + fp2)

    PR_for_FPR_M += [Pr2]
    PR_for_TPR_M += [Pr1]
    TPR_for_TPR_M += [optim_tpr]
    FPR_for_TPR_M += [fpr_optim_tpr]
    FPR_for_FPR_M += [optim_fpr]
    TPR_for_FPR_M += [tpr_optim_fpr]

print(f"Computed metrics for {len(AP_M)} multi-proxy folds")

results/test_tc_M_flowlen-100_neg-49_NtimesN-fold0-run10.pkl
results/test_tc_M_flowlen-100_neg-49_NtimesN-fold1-run10.pkl
results/test_tc_M_flowlen-100_neg-49_NtimesN-fold2-run10.pkl
results/test_tc_M_flowlen-100_neg-49_NtimesN-fold3-run10.pkl
results/test_tc_M_flowlen-100_neg-49_NtimesN-fold4-run10.pkl
Computed metrics for 5 multi-proxy folds


In [20]:
AP_S = []
PR_for_FPR_S = []
PR_for_TPR_S = []
TPR_for_TPR_S = []
FPR_for_TPR_S = []
FPR_for_FPR_S = []
TPR_for_FPR_S = []

for resfile in S_resfiles:
    print(resfile)
    f = open(resfile, "rb")
    props, y_truth = pickle.load(f)

    av_prec = average_precision_score(y_truth, props)

    AP_S += [av_prec]

    pr, rec, _ = plot_pr(props, y_truth, plot=False)
    fpr, tpr, thresholds = plot_roc(props, y_truth, plot=False)
    tpr_optim_fpr, optim_fpr, optim_tpr, fpr_optim_tpr = optimize(fpr, tpr, thresholds, pr, rec)

    n_pos = len([y for y in y_truth if y == 1]) # 1840
    n_neg = len([y for y in y_truth if y == 0])
    print(n_pos, n_neg)

    tp1 = optim_tpr * n_pos
    fp1 = fpr_optim_tpr * n_neg
    Pr1 = tp1 / (tp1 + fp1)

    tp2 = tpr_optim_fpr * n_pos
    fp2 = optim_fpr * n_neg
    Pr2 = tp2 / (tp2 + fp2)

    PR_for_FPR_S += [Pr2]
    PR_for_TPR_S += [Pr1]
    TPR_for_TPR_S += [optim_tpr]
    FPR_for_TPR_S += [fpr_optim_tpr]
    FPR_for_FPR_S += [optim_fpr]
    TPR_for_FPR_S += [tpr_optim_fpr]

print(f"Computed metrics for {len(AP_S)} single-proxy folds")

results/test_tc_S_flowlen-100_neg-49_NtimesN-fold0-run10.pkl
1590 2526510
results/test_tc_S_flowlen-100_neg-49_NtimesN-fold1-run10.pkl
1590 2526510
results/test_tc_S_flowlen-100_neg-49_NtimesN-fold2-run10.pkl
1590 2526510
results/test_tc_S_flowlen-100_neg-49_NtimesN-fold3-run10.pkl
1590 2526510
results/test_tc_S_flowlen-100_neg-49_NtimesN-fold4-run10.pkl
1590 2526510
Computed metrics for 5 single-proxy folds


In [21]:
AP_S = [round(ap*100, 2) for ap in AP_S]
AP_M = [round(ap*100, 2) for ap in AP_M]

PR_for_FPR_S = [round(pr*100, 4) for pr in PR_for_FPR_S]
PR_for_FPR_M = [round(pr*100, 4) for pr in PR_for_FPR_M]
PR_for_TPR_S = [round(pr*100, 4) for pr in PR_for_TPR_S]
PR_for_TPR_M = [round(pr*100, 4) for pr in PR_for_TPR_M]
FPR_for_TPR_S = [round(fpr, 4) for fpr in FPR_for_TPR_S]
FPR_for_TPR_M = [round(fpr, 4) for fpr in FPR_for_TPR_M]

In [25]:
metric_array = PR_for_TPR_S

print("SINGLE:")
for metric in metric_array:
    print(metric)
        
metric_array = PR_for_TPR_M

print("\nMULTI:")
for metric in metric_array:
    print(metric)

SINGLE:
11.8764
7.7468
9.6811
12.0069
9.4761

MULTI:
4.7395
4.5624
7.1166
2.0596
5.2467


## Plot convergence graphs

In [52]:
# BUILD MAIN PLOTS for multi

import pickle

M_dir = "log_tc_M_neg-49/train/"

results = {}

for fold in folds:
    results[fold] = {}
    for run in runs:
        f = open(M_dir + f"train_last_aux_tc_M_fold{fold}-{run}.pkl", "rb")
        results[fold][run] = pickle.load(f)
                
print(len(results))

av_prec_M = []
loss_M = []
best_AP_M = []
best_AP_epochs_M = []

for res in results:
    for fold in results[res]:
        val_av_prec = results[res][fold]["history"]["val_av_prec"]
        val_loss = results[res][fold]["history"]["val_loss"]
        best_AP = results[res][fold]["best_av_prec"]
        best_AP_epoch = results[res][fold]["best_av_prec"]
        av_prec_M += [(val_av_prec, list(range(len(val_av_prec))))]
        loss_M += [(val_loss, list(range(len(val_loss))))]
        best_AP_M += [best_AP]
        best_AP_epochs_M += [best_AP_epoch]

2


In [53]:
# BUILD MAIN PLOTS for single

S_dir = "log_tc_S_neg-49/train/"

results = {}

for fold in folds:
    results[fold] = {}
    for run in runs:
        f = open(S_dir + f"train_last_aux_tc_S_fold{fold}-{run}.pkl", "rb")
        results[fold][run] = pickle.load(f)
            
print(len(results))

av_prec_S = []
loss_S = []
best_AP_S = []
best_AP_epochs_S = []

for res in results:
    for fold in results[res]:
        val_av_prec = results[res][fold]["history"]["val_av_prec"]
        val_loss = results[res][fold]["history"]["val_loss"]
        best_AP = results[res][fold]["best_av_prec"]
        best_AP_epoch = results[res][fold]["best_av_prec"]
        av_prec_S += [(val_av_prec, list(range(len(val_av_prec))))]
        loss_S += [(val_loss, list(range(len(val_loss))))]
        best_AP_S += [best_AP]
        best_AP_epochs_S += [best_AP_epoch]

2


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

SMALLER_SIZE = 11
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

LINE_WIDTH=1.5
fig, ax1 = plt.subplots(figsize=(6,4))

plots = []
lab_lns = None
made_label = False
for score in av_prec_S:
    if not made_label:
        p = ax1.plot(score[1], score[0], color='#8DB600',
                     label=f'Single-proxy dataset SD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)
        plots+=[p]
        lab_lns = p 
        made_label = True
    
    else:
        plots+=[ax1.plot(score[1], score[0], color='#8DB600',
                     #label=f'Single-proxy dataset SD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)]
made_label = False    
for score in av_prec_M:
    if not made_label:
        p = ax1.plot(score[1], score[0], color='#FE6F5E',
                     label=f'Multi-proxy dataset MD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)
        plots += [p]
        lab_lns += p
        made_label = True
    else:
        plots+=[ax1.plot(score[1], score[0], color='#FE6F5E',
                     #label=f'Multi-proxy dataset MD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)]
        
labs = [l.get_label() for l in lab_lns]
ax1.legend(lab_lns, labs, loc=7)

lns = plots[0]
for i in range(1, len(plots)):
    lns += plots[i]

ax1.set_xlabel('Number of epochs')
ax1.set_ylabel('Average Precision')
ax1.set_ylim([0.0, 1.0])

#plt.title('Convergence Speed')
plt.legend()
#ax1.grid(color='silver', linewidth=1)
ax1.yaxis.grid(linestyle='--', color='silver', linewidth=1, zorder=0)
#fig.savefig("figures/AP_convspeed.pdf", dpi=300, bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

SMALLER_SIZE = 11
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

LINE_WIDTH=1.5
fig, ax1 = plt.subplots(figsize=(6,4))

plots = []
lab_lns = None
made_label = False
for score in loss_S:
    if not made_label:
        p = ax1.plot(score[1], score[0], color='#FFBF00',
                     label=f'Single-proxy dataset SD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)
        plots+=[p]
        lab_lns = p 
        made_label = True
    
    else:
        plots+=[ax1.plot(score[1], score[0], color='#FFBF00',
                     #label=f'Single-proxy dataset SD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)]
made_label = False    
for score in loss_M:
    if not made_label:
        p = ax1.plot(score[1], score[0], color='#21ABCD',
                     label=f'Multi-proxy dataset MD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)
        plots += [p]
        lab_lns += p
        made_label = True
    else:
        plots+=[ax1.plot(score[1], score[0], color='#21ABCD',
                     #label=f'Multi-proxy dataset MD',
                     #markevery=5, marker='o',
                     linewidth=LINE_WIDTH)]
        
labs = [l.get_label() for l in lab_lns]
ax1.legend(lab_lns, labs, loc=1)

lns = plots[0]
for i in range(1, len(plots)):
    lns += plots[i]

ax1.set_xlabel('Number of epochs')
ax1.set_ylabel('Loss')
#ax1.set_ylim([0.01, 0.12])

#plt.title('Convergence Speed')
plt.legend()
#ax1.grid(color='silver', linewidth=1)
ax1.yaxis.grid(linestyle='--', color='silver', linewidth=1, zorder=0)
#fig.savefig("figures/Loss_convspeed_new_run0.pdf", dpi=300, bbox_inches = 'tight', pad_inches = 0)
plt.show()